<a href="https://colab.research.google.com/github/GregWick1/deep-learning-challenge/blob/main/Alphabet_Soup_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df['APPLICATION_TYPE'].value_counts()
application_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_replace = list(application_count[application_count < 500].index)

# Replace in dataframe
for app in application_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df['CLASSIFICATION'].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_count_1 = classification_count.loc[classification_count > 1]
classification_count_1.head()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_count[classification_count < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies_df = pd.get_dummies(application_df)
application_dummies_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [11]:
# Split our preprocessed data into our features and target arrays
X = application_dummies_df.drop('IS_SUCCESSFUL', axis=1).values
y = application_dummies_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
total_input_features = len(X_train_scaled[0])
hidden_node1 = 33
hidden_node2 = 17
hidden_node3 = 5


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node1, input_dim = total_input_features, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node2, activation = "relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node3, activation = "relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 33)                1452      
                                                                 
 dense_8 (Dense)             (None, 17)                578       
                                                                 
 dense_9 (Dense)             (None, 5)                 90        
                                                                 
 dense_10 (Dense)            (None, 1)                 6         
                                                                 
Total params: 2126 (8.30 KB)
Trainable params: 2126 (8.30 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.4 MB/s eta 0:00:00


In [24]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 6s 5ms/step - loss: 0.5878 - accuracy: 0.7079
Epoch 2/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5572 - accuracy: 0.7288
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5533 - accuracy: 0.7296
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5517 - accuracy: 0.7315
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5501 - accuracy: 0.7317
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5492 - accuracy: 0.7309
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7326
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5477 - accuracy: 0.7315
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7327
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5469 - accura

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5539 - accuracy: 0.7292 - 460ms/epoch - 2ms/step
Loss: 0.5539449453353882, Accuracy: 0.7292128205299377


In [28]:
# Changing hidden nodes
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
total_input_features = len(X_train_scaled[0])
hidden_node1 = 60
hidden_node2 = 30
hidden_node3 = 15
hidden_node4 = 7


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node1, input_dim = total_input_features, activation = "relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node2, activation = "relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node3, activation = "relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_node4, activation = "sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 60)                2640      
                                                                 
 dense_15 (Dense)            (None, 30)                1830      
                                                                 
 dense_16 (Dense)            (None, 15)                465       
                                                                 
 dense_17 (Dense)            (None, 7)                 112       
                                                                 
 dense_18 (Dense)            (None, 1)                 8         
                                                                 
Total params: 5055 (19.75 KB)
Trainable params: 5055 (19.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:

# Compile the model (2nd Attempt)
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [30]:

# Train the model (2nd Attempt)
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5797 - accuracy: 0.7067
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5547 - accuracy: 0.7299
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5524 - accuracy: 0.7301
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5506 - accuracy: 0.7332
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5495 - accuracy: 0.7333
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5477 - accuracy: 0.7341
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5473 - accuracy: 0.7345
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5474 - accuracy: 0.7341
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5461 - accuracy: 0.7346
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5458 - accura

In [31]:
# Evaluate the Model
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5530 - accuracy: 0.7301 - 685ms/epoch - 3ms/step
Loss: 0.5530264377593994, Accuracy: 0.7301457524299622


In [32]:
# Optimizer: Keras Tuner
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=30,
        step=5), activation=activation, input_dim=total_input_features ))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [33]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [34]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=100,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 51s]
val_accuracy: 0.7315452098846436

Best val_accuracy So Far: 0.7332944869995117
Total elapsed time: 00h 23m 01s


In [35]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'sigmoid', 'first_units': 11, 'num_layers': 1, 'units_0': 1, 'units_1': 21, 'units_2': 26, 'units_3': 1, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0014'}
{'activation': 'sigmoid', 'first_units': 26, 'num_layers': 5, 'units_0': 21, 'units_1': 26, 'units_2': 21, 'units_3': 16, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'units_4': 1, 'tuner/trial_id': '0020'}
{'activation': 'tanh', 'first_units': 16, 'num_layers': 4, 'units_0': 1, 'units_1': 1, 'units_2': 1, 'units_3': 16, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}


In [36]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5711 - accuracy: 0.7333 - 1s/epoch - 5ms/step
Loss: 0.5711050629615784, Accuracy: 0.7332944869995117
268/268 - 1s - loss: 0.5731 - accuracy: 0.7333 - 1s/epoch - 4ms/step
Loss: 0.5731104016304016, Accuracy: 0.7332944869995117
268/268 - 1s - loss: 0.5743 - accuracy: 0.7332 - 851ms/epoch - 3ms/step
Loss: 0.5743119716644287, Accuracy: 0.7331778407096863


In [37]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
